In [1]:
import numpy as np
def b():
    a = [2,3,4,5]
    a = np.asarray(a)
    a = list(a)
    return tuple([a[0], a[-1]] + a[1:-1])
b()

(2, 5, 3, 4)

In [2]:
import numpy as np
import pickle
from pathlib import Path

from pathlib import Path
repo_root = Path().resolve().parent
# print(repo_root)
import sys ; sys.path.insert(0, str(repo_root))
import os
datapath = Path('/home/jseia/Desktop/thesis/data/')
base_path = Path('/home/jseia/Desktop/thesis/code/nnUNet/nnunetv2')
os.environ['nnUNet_raw'] = str(base_path/'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = str(base_path/'preprocessed')
os.environ['nnUNet_results'] = str(base_path/'nnUNet_trained_models')

import argparse
import datetime
import json
import math

import sys
import shutil
import time
import torch

import numpy as np
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

from pathlib import Path
from typing import Union, Optional

import SSL.desd_utils as utils
from SSL.models import DINOHead
from SSL.data_loader_ssl import Dataset3D
from nnunetv2.run.run_training import get_trainer_from_args, maybe_load_checkpoint
from torch.backends import cudnn

def run_training_our(dataset_name_or_id: Union[str, int],
                 configuration: str, fold: Union[int, str],
                 trainer_class_name: str = 'nnUNetTrainer',
                 plans_identifier: str = 'nnUNetPlans',
                 pretrained_weights: Optional[str] = None,
                 use_compressed_data: bool = False,
                 export_validation_probabilities: bool = False,
                 continue_training: bool = False,
                 only_run_validation: bool = False,
                 disable_checkpointing: bool = False,
                 device: torch.device = torch.device('cuda')):
    if isinstance(fold, str):
        if fold != 'all':
            try:
                fold = int(fold)
            except ValueError as e:
                print(f'Unable to convert given value for fold to int: {fold}. fold must bei either "all" or an integer!')
                raise e

    nnunet_trainer = get_trainer_from_args(dataset_name_or_id, configuration, fold, trainer_class_name,
                                            plans_identifier, use_compressed_data, device=device)

    if disable_checkpointing:
        nnunet_trainer.disable_checkpointing = disable_checkpointing

    assert not (continue_training and only_run_validation), f'Cannot set --c and --val flag at the same time. Dummy.'

    maybe_load_checkpoint(nnunet_trainer, continue_training, only_run_validation, pretrained_weights)

    if torch.cuda.is_available():
        cudnn.deterministic = False
        cudnn.benchmark = True

    nnunet_trainer.initialize()
    return nnunet_trainer.network

In [7]:
from torchsummary import summary
class res3d(nn.Module):
    def __init__(self,teacher=False):
        super(res3d, self).__init__()

        self.teacher = teacher
        self.net = run_training_our(
            'Dataset002_AIS',
            '3d_fullres',
            0,
            'nnUNetTrainer',
            'nnUNetPlans',
            None,
            False,
            False,
            False,
            False,
            False,
            torch.device('cpu')
        )
        # print(summary(self.net, (1,96,160,160)), 2, 'cpu')
        self.net = self.net.encoder
        model = self.net.stages
        n_mods = model.named_modules()
        self.n_stages = len([
            name for name, _ in n_mods if ('.' not in name) and (name != '')])

        self.gap = nn.AdaptiveAvgPool3d((1, 1, 1))

    def forward(self, x):
        # first layer/block
        x = self.net(x)
        out = []
        for i in range(self.n_stages):
            print(x[i].size())
            out.append(self.gap(x[i]).flatten(1))
        if self.teacher:
            return out[-1]
        return out

In [8]:
student = res3d(False)
teacher = res3d(True)

Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



In [5]:
# import pickle

# with open('/home/jseia/Desktop/thesis/code/nnUNet_ais/nnunetv2/preprocessed/nnUNetPlans_3d_fullres/strokeperf0001.pkl', 'rb') as file:
#     a = pickle.load(file)

# a

In [11]:
from torchsummary import summary
# model = student.net.stages
random_sample = torch.rand([2,1,96,160,160])
random_return = student(random_sample)
for i in range(len(random_return)):
    print(random_return[i].size())
# for name, module in model.named_modules():
    

torch.Size([2, 32, 96, 160, 160])
torch.Size([2, 64, 48, 80, 80])
torch.Size([2, 128, 24, 40, 40])
torch.Size([2, 256, 12, 20, 20])
torch.Size([2, 320, 6, 10, 10])
torch.Size([2, 320, 6, 5, 5])
torch.Size([2, 32])
torch.Size([2, 64])
torch.Size([2, 128])
torch.Size([2, 256])
torch.Size([2, 320])
torch.Size([2, 320])
